In [51]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

from Modules.Utils.Imports import *
from Modules.Models.BuildBINNs import BINNCovasim
from Modules.Utils.ModelWrapper import ModelWrapper

import Modules.Utils.PDESolver as PDESolver
import Modules.Loaders.DataFormatter as DF
from utils import get_case_name_drums, AIC_OLS, RSS

In [52]:
# helper functions
def to_torch(x):
    return torch.from_numpy(x).float().to(device)
def to_numpy(x):
    return x.detach().cpu().numpy()

In [53]:
device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))
# choose and get data
path = '../Data/covasim_data/drums_data/'
population = 200000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic = True
chi_type = 'piecewise'
date = '0623_1017'
case_name = get_case_name_drums(date)
params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name, plot=False)

Device set to cpu


In [54]:
comps = list(params.keys())[1:]
for comp in comps:
    if comp=='S':
        data = np.array(params[comp])[:,None]
    else:
        new_col = np.array(params[comp])[:,None]
        data = np.append(data, new_col, 1)

In [56]:
data = (data / 50000.)
N = len(data)
t_max = N - 1
t = np.arange(N)

In [ ]:
# instantiate BINN model
binn = BINNCovasim(params, t_max, tracing_array=None, keep_d=keep_d).to(device)